 # Daily Profit for Trades in TradeBench

In [138]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys  # Only needed to determine Python version
import matplotlib  # Only needed to determine Matplotlib version number
import chardet
from datetime import date
from pandas import DataFrame
from scipy import stats
import seaborn as sns

# Enable inline plotting
# %matplotlib inline

print('Python Version: ' + sys.version)
print('Pandas Version: ' + pd.__version__)
print('Matplotlib Version: ' + matplotlib.__version__)

Python Version: 3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas Version: 1.0.1
Matplotlib Version: 3.1.3


In [139]:
#/Users/<USERNAME>/<CSV_LOCATION>.csv
CSV_Location = r'/Users/Pho/Documents/Dev/DayTrading/TradingAnalysisScripts/TradeBenchCSV/2020-08-19-TradeBench-closed-trades-export-3.csv'

df = pd.read_csv(CSV_Location, encoding = "ISO-8859-1")

del df["Post Trade Analysis"]

print(df.columns)

Index(['Symbol', 'Name', 'Long/Short', 'Plan Date', 'Entry (first) Date',
       'Entry (first) Time', 'Exit (last) Date', 'Exit (last) Time',
       'Quantity', 'Avg. Entry', 'Avg. Exit', 'Entry Commission',
       'Exit Commission', 'Net P/L Amount', 'Net P/L %', 'Broker', 'Strategy',
       'Planned Stop Loss Price', 'First Stop Loss Price',
       'Trade Plan Description', 'Label 1', 'Label 2', 'Label 3', 'Label 4'],
      dtype='object')


In [140]:
# Combine dates and times together
df["Entry Date"] = df["Entry (first) Date"].astype(str) + " " + df["Entry (first) Time"].astype(str)
df["Exit Date"] = df["Exit (last) Date"].astype(str) + " " + df["Exit (last) Time"].astype(str)

df["Entry Date"] = df["Entry Date"].astype(str)
#print(df["Entry Date"])

Location2 = f'/Users/Pho/Documents/Dev/DayTrading/TradingAnalysisScripts/TradeBenchCSV/ting.csv'
df.to_csv(Location2)

df["Entry Date"] = pd.to_datetime(df["Entry Date"], format="%d/%m/%Y %H:%M:%S")
df["Exit Date"] = pd.to_datetime(df["Exit Date"], format="%d/%m/%Y %H:%M:%S")

#print(df.info(verbose = True))
print(df["Entry Date"])

0    2020-08-10 14:37:00
1    2020-08-10 15:26:00
2    2020-08-11 14:41:00
3    2020-08-11 14:46:00
4    2020-08-11 14:54:00
5    2020-08-11 14:59:00
6    2020-08-12 14:37:00
7    2020-08-12 14:54:00
8    2020-08-12 15:06:00
9    2020-08-12 15:06:00
10   2020-08-14 14:30:00
11   2020-08-17 14:31:00
12   2020-08-17 14:31:00
13   2020-08-17 15:15:00
14   2020-08-17 15:33:00
15   2020-08-10 14:36:00
16   2020-08-18 14:33:00
17   2020-08-18 14:49:00
18   2020-08-18 14:54:00
19   2020-08-18 14:59:00
20   2020-08-18 15:05:00
21   2020-08-10 14:42:00
22   2020-08-10 14:46:00
23   2020-08-10 14:36:00
Name: Entry Date, dtype: datetime64[ns]


In [144]:
#Group by hour, day, month live paper trades only

df_Live_PaperTrades = df[df["Broker"] == "PaperTrade Broker"]

byHour = df_Live_PaperTrades.groupby(df_Live_PaperTrades["Entry Date"].dt.hour)["Net P/L Amount"].agg([np.ma.count, np.sum, np.mean, np.median, stats.mode, np.ptp, stats.iqr, np.std, "min", "max"])

byDay = df_Live_PaperTrades.groupby(df_Live_PaperTrades["Entry Date"].dt.day)["Net P/L Amount"].agg([np.ma.count, np.sum, np.mean, np.median, stats.mode, np.ptp, stats.iqr, np.std, "min", "max"])

byMonth = df_Live_PaperTrades.groupby(df_Live_PaperTrades["Entry Date"].dt.month)["Net P/L Amount"].agg([np.ma.count, np.sum, np.mean, np.median, stats.mode, np.ptp, stats.iqr, np.std, "min", "max"])

print(byDay)

count    sum    mean  median             mode    ptp      iqr  \
Entry Date                                                                  
10            2.0  -6.92 -3.4600  -3.460  ([-20.55], [1])  34.18  17.0900   
11            4.0  16.21  4.0525   4.790    ([0.62], [1])   5.39   1.8725   
12            4.0  23.69  5.9225   8.370  ([-27.09], [1])  61.13  28.3625   
14            1.0   1.05  1.0500   1.050    ([1.05], [1])   0.00   0.0000   
17            4.0 -26.93 -6.7325  -0.765   ([-25.7], [1])  26.00   7.1825   
18            5.0  20.35  4.0700  -2.950  ([-28.39], [1])  73.84  55.0600   

                  std    min    max  
Entry Date                           
10          24.168910 -20.55  13.63  
11           2.376740   0.62   6.01  
12          26.105403 -27.09  34.04  
14                NaN   1.05   1.05  
17          12.660414 -25.70   0.30  
18          32.912168 -28.39  45.45  
